# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)

/home/workspace
['/home/workspace/event_data/2018-11-27-events.csv', '/home/workspace/event_data/2018-11-04-events.csv', '/home/workspace/event_data/2018-11-07-events.csv', '/home/workspace/event_data/2018-11-09-events.csv', '/home/workspace/event_data/2018-11-19-events.csv', '/home/workspace/event_data/2018-11-05-events.csv', '/home/workspace/event_data/2018-11-22-events.csv', '/home/workspace/event_data/2018-11-16-events.csv', '/home/workspace/event_data/2018-11-26-events.csv', '/home/workspace/event_data/2018-11-24-events.csv', '/home/workspace/event_data/2018-11-29-events.csv', '/home/workspace/event_data/2018-11-15-events.csv', '/home/workspace/event_data/2018-11-20-events.csv', '/home/workspace/event_data/2018-11-06-events.csv', '/home/workspace/event_data/2018-11-18-events.csv', '/home/workspace/event_data/2018-11-21-events.csv', '/home/workspace/event_data/2018-11-10-events.csv', '/home/workspace/event_data/2018-11-23-events.csv', '/home/workspace/event_data/2018-11-02-events.c

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))

csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
from cassandra.cluster import Cluster
try:
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()
except Exception as connection_error:
    print(connection_error)

#### Create Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS project_2
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as keyspace_error:
    print(keyspace_error)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('project_2')
except Exception as keyspace_set_error:
    print(keyspace_set_error)

In [12]:
from prettytable import PrettyTable
file = 'event_datafile_new.csv'
    

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




# Query 1 Section

## Create Table

In [54]:
#Query 1:  Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

query = "CREATE TABLE IF NOT EXISTS artist_song_duration"
query = query + "(sessionId int, itemInSession int, artist text, song text, length float, PRIMARY KEY (sessionId, itemInSession, artist, song, length))"
try:
    session.execute(query)
except Exception as e:
    print(e)
                   

## Insert Data

In [55]:


with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "insert into artist_song_duration (sessionId, itemInSession, artist, song, length)"
        query = query + "values (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

## Test Inserted Data

In [56]:
query = "select * from artist_song_duration limit 10"
final = PrettyTable()
final.field_names = ["sessionid", "iteminsession", "artist", "song", "length"]
try:
    rows = session.execute(query)
except Exception as select_error:
    print(select_error)
    
for row in rows:
    final.add_row([row.sessionid, row.iteminsession, row.artist, row.song, row.length])
    
print(final)
    

+-----------+---------------+--------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+
| sessionid | iteminsession |       artist       |                                                                           song                                                                          |       length       |
+-----------+---------------+--------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+
|     23    |       0       |   Regina Spektor   |                                                             The Calculation (Album Version)                                                             | 191.08526611328125 |
|     23    |       1       |  Octopus Project   |                                              

## Select Data 

In [57]:
# query1

query = "select artist, song, length from artist_song_duration where sessionId = 338 and itemInSession = 4"
final = PrettyTable()
final.field_names = ["artist", "song", "length"]
try:
    rows = session.execute(query)
except Exception as select_error:
    print(select_error)
    
for row in rows:
    final.add_row([row.artist, row.song, row.length])
    
print(final)

+-----------+---------------------------------+--------------------+
|   artist  |               song              |       length       |
+-----------+---------------------------------+--------------------+
| Faithless | Music Matters (Mark Knight Dub) | 495.30731201171875 |
+-----------+---------------------------------+--------------------+


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

# Query 2 Section

## Create Table

In [58]:
##Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

query = "CREATE TABLE IF NOT EXISTS song_playlist_session "
query = query + "(userId int, sessionId int, itemInSession int, artist text, song text, firstName text, lastName text, PRIMARY KEY ((userId, sessionId), itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)

## Insert Data

In [59]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "insert into song_playlist_session (userId, sessionId,itemInSession, artist, song, firstName, lastName)"
        query = query + "values (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

## Select Query

In [60]:
##Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

query = "select itemInSession, artist, song, firstName, lastName from song_playlist_session where userId = 10 and sessionId = 182 "
final = PrettyTable()
final.field_names = ["itemsession", "artist", "song", "firstname", "lastname"]
try:
    rows = session.execute(query)
except Exception as select_error:
    print(select_error)
    
for row in rows:
    final.add_row([row.iteminsession, row.artist, row.song, row.firstname, row.lastname])
    
print(final)

+-------------+-------------------+------------------------------------------------------+-----------+----------+
| itemsession |       artist      |                         song                         | firstname | lastname |
+-------------+-------------------+------------------------------------------------------+-----------+----------+
|      0      |  Down To The Bone |                  Keep On Keepin' On                  |   Sylvie  |   Cruz   |
|      1      |    Three Drives   |                     Greece 2000                      |   Sylvie  |   Cruz   |
|      2      | Sebastien Tellier |                      Kilometer                       |   Sylvie  |   Cruz   |
|      3      |   Lonnie Gordon   | Catch You Baby (Steve Pitron & Max Sanna Radio Edit) |   Sylvie  |   Cruz   |
+-------------+-------------------+------------------------------------------------------+-----------+----------+


# Query 3 Section

## Create Table

In [20]:
##Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
query = "CREATE TABLE IF NOT EXISTS user_sessions"
query = query + "(song text, userId int,firstname text, lastname text,  PRIMARY KEY (song, userId))"
try:
    session.execute(query)
except Exception as e:
    print(e)             

## Insert Data

In [21]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "insert into user_sessions (song, userId,firstname, lastname)"
        query = query + "values (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]),line[1], line[4]))

## Select Query

In [22]:
##Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
query = "select firstname, lastname from user_sessions where song = 'All Hands Against His Own'"
final = PrettyTable()
final.field_names = ["firstname", "lastname"]
try:
    rows = session.execute(query)
except Exception as select_error:
    print(select_error)
    
for row in rows:
    final.add_row([row.firstname, row.lastname])
print(final)

+------------+----------+
| firstname  | lastname |
+------------+----------+
| Jacqueline |  Lynch   |
|   Tegan    |  Levine  |
|    Sara    | Johnson  |
+------------+----------+


### Drop the tables before closing out the sessions

In [13]:
query = " drop table artist_song_duration"
try:
    rows = session.execute(query)
except Exception as drop_error:
    print(drop_error)

In [34]:
query = " drop table song_playlist_session"
try:
    rows = session.execute(query)
except Exception as drop_error:
    print(drop_error)

In [19]:
query = " drop table user_sessions"
try:
    rows = session.execute(query)
except Exception as drop_error:
    print(drop_error)

### Close the session and cluster connection¶

In [41]:
session.shutdown()
cluster.shutdown()